# README
LabelEncodingを3つの方法でやってみる

1. 手動
2. sklean.preprocessing.LabelEncoder
3. category_encoders

In [1]:
import pandas as pd
import category_encoders
from sklearn.preprocessing import LabelEncoder
import io

In [2]:
train = pd.read_csv('../input/train.csv', usecols=['Sex', 'Embarked', 'Cabin'])
train = train.loc[:, ['Sex', 'Embarked', 'Cabin']]  # わかりやすくするために加工順序に並べ替え

train.head()

,Sex,Embarked,Cabin
0,male,S,NaN
1,female,C,C85
2,female,S,NaN
3,female,S,C123
4,male,S,NaN


In [3]:
train.isnull().sum()

Sex           0
Embarked      2
Cabin       687
dtype: int64

# 手動
- Cabinはカーディナリティが大きすぎてムリ！

In [4]:
data = train.copy()

In [5]:
# SexのEncodingは簡単
# 狭義にはBinaryEncodingって感じっぽいね
data['SexLabel'] = data['Sex'].map({'male': 1, 'female': 0})

data.head()

,Sex,Embarked,Cabin,SexLabel
0,male,S,NaN,1
1,female,C,C85,0
2,female,S,NaN,0
3,female,S,C123,0
4,male,S,NaN,1


In [6]:
# EmbarkedのEncodingも簡単
data['EmbarkedLabel'] = data['Embarked'].map({'S': 1, 'C': 2, 'Q': 3})

data.head()

,Sex,Embarked,Cabin,SexLabel,EmbarkedLabel
0,male,S,NaN,1,1.0
1,female,C,C85,0,2.0
2,female,S,NaN,0,1.0
3,female,S,C123,0,1.0
4,male,S,NaN,1,1.0


In [7]:
# Cabinはカーディナリティが大きすぎて手動LabelEncodingムリだ！

# sklearn
- ndarrayを返す。なので、自分で追加する必要がある
- 欠損があるとエラーになる

In [8]:
data = train.copy()

In [9]:
label_encoder = LabelEncoder()

data['SexLabel'] = label_encoder.fit_transform(data['Sex'])

data.head()

,Sex,Embarked,Cabin,SexLabel
0,male,S,NaN,1
1,female,C,C85,0
2,female,S,NaN,0
3,female,S,C123,0
4,male,S,NaN,1


In [10]:
# TypeError! 欠損があるためそのままでは対応できない
# label_encoder.fit_transform(data['Embarked'])

In [11]:
# Cabinも同じく
# label_encoder.fit_transform(data['Cabin'])

# category_encoders
- 欠損にどう対応するか？
- DFを返す
- 元のカラムを返すかどうか
- まとめて対応できる(わかりにくいといえばわかりにくくなる)
- LabelEncoderはないので、OrdinalEncoderでやや強引に対応する
- どっちがいいかでいうと、微妙なところ

##  handle_missing='return_nan

In [12]:
data = train.copy()


encoder = category_encoders.OrdinalEncoder(cols=['Sex', 'Embarked'],
                                           handle_missing='return_nan')  # NaNをそのままにする。Sexもfloatになっちまう

data = encoder.fit_transform(data)

data[data['Embarked'].isnull()]

,Sex,Embarked,Cabin
61,2.0,NaN,B28
829,2.0,NaN,B28


## handle_missing='value'  # NaNを第4の値として処理する
- 欠損に背景因子がある場合は有効だと思う
- ランダムな欠損であれば意味はなさそう

In [13]:
data = train.copy()


encoder = category_encoders.OrdinalEncoder(cols=['Sex', 'Embarked'],
                                           handle_missing='value')  # NaNを第4の値として処理する

data = encoder.fit_transform(data)

data['Embarked'].value_counts()

1    644
2    168
3     77
4      2
Name: Embarked, dtype: int64

## handle_missing='error'  # 欠損があるとエラー

In [14]:
data = train.copy()


encoder = category_encoders.OrdinalEncoder(cols=['Sex', 'Embarked'],
                                           handle_missing='error')  # 欠損があるとエラー

# ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
# data = encoder.fit_transform(data)

# おわりに
- category_encordersがよさそうだね
- とはいえ、これくらいの処理ならどれでもって感じもある